In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score

# Load the data
data = pd.read_csv('merged_results.csv')

# Prepare the data
y = data['eTime_windows10'].values
X = data.drop(['eTime_windows10', 'Filename'], axis=1)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the model
def create_model():
    model = Sequential([
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics = ['r2_score'])
    return model

# Train the model
model = create_model()
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
mse, r2_score = model.evaluate(X_test_scaled, y_test)
print(f"Model MSE: {mse}, R2:{r2_score}")

# Make predictions
predictions = model.predict(X_test_scaled)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.4831 - r2_score: -496468.8125 - val_loss: 0.0582 - val_r2_score: -76400.6484
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1583 - r2_score: -174149.0625 - val_loss: 0.0779 - val_r2_score: -102324.1094
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0995 - r2_score: -109584.5547 - val_loss: 0.0928 - val_r2_score: -121890.6875
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0626 - r2_score: -73472.3594 - val_loss: 0.0847 - val_r2_score: -111202.0312
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0531 - r2_score: -58042.8555 - val_loss: 0.0589 - val_r2_score: -77330.4453
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0381 - r2_score: -41529.5000 - val_loss: 0.0411 - val_r2_score: -53931.7656
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0168 - r2_score: -18730.0371 - val_loss: 0.0318 - val_r2_score: -41789.1797
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

In [7]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

# 定义基学习器
base_learners = [
    ('xgb', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('svr', SVR(kernel='rbf', C=1, gamma=0.1))
]

# 定义堆叠回归模型
stacking_model = StackingRegressor(estimators=base_learners, final_estimator=RidgeCV())

# 训练模型
stacking_model.fit(X_train_scaled, y_train)

# 预测和评估模型
y_pred = stacking_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 1.0788477367559471e-06
R-squared: -0.033734659772433506
